### Memory

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory

# 채팅 기록을 저장할 클래스 정의
# BaseChatMessageHistory를 상속받는다.
class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []  # 메세지들을 저장할 빈 리스트 생성

    def add_messages(self, messages):
        # 메세지 리스트가 들어오면 기존 리스트에 추가
        self.messages.extend(messages)

    def clear(self):
        # 기록 초기화
        self.messages = []

# 세션(사용자)별로 기록을 관리할 딕셔너리 생성
# store = {'user1' : History객체, 'user2' : History객체, ...}
store = {}

# 세션 ID를 받아 해당 세션의 History 객체를 반환하는 함수
def get_by_session_id(session_id):
    if session_id not in store:
        # 새로운 InMemoryHistory 객체를 만들어서 store에 저장
        store[session_id] = InMemoryHistory()
    # 해당하는 session_id 기록 객체를 반환
    return store[session_id]

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# 프롬프트 템플릿 생성
prompts = ChatPromptTemplate.from_messages([
    ('system', '너는 {skill}을 잘하는 도우미야'), # 시스템 메세지 (페르소나 설정)
    MessagesPlaceholder(variable_name="history"), # 대화 기록이 주입될 위치
    ('human', '{query}') # 사용자 질문
])

# 모델 생성
model = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.8)

# 기본 체인 생성
chain = prompts | model

#  메모리 기능 장착한 체인 생성(RunnableWithMessageHistory)
#  기존 chain을 감싸서, 대화 기록을 자동으로 읽고 쓰는 기능 추가
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id, # 세션 ID로 기록을 찾아오는 함수 연결
    input_messages_key='query',  # 사용자 입력이 들어오는 변수명
    history_messages_key='history'  # 프롬프트에서 기록이 들어갈 변수명 (Placeholder 이름과 일치해야 함)
)

In [ ]:
response = chain_with_history.invoke(
    {'skill' : 'math', 'query' : '철수는 강아지를 세마리 키우고 있습니다.'},
    config={'configurable':{'session_id': 'abc'}}   # session_id를 'abc'로 설정
)

print(response.content)

In [ ]:
response2 = chain_with_history.invoke(
    {'skill':'math', 'query': '영희가 고양이를 두마리 키우고 있습니다.'},
    config={'configurable': {'session_id' : 'abc'}}
)

print(response2.content)

In [ ]:
# 문맥을 파악해야만 풀 수 있는 문제
response3 = chain_with_history.invoke(
    {'skill':'math', 'query': '철수랑 영희랑 합해서 몇 마리의 동물을 키우고 있나요??'},
    config={'configurable': {'session_id' : 'abc'}}
)

print(response3.content)

In [ ]:
# 저장소 확인
print(store['abc'])